In [2]:
import torch
import torchvision.models as models
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from sklearn.linear_model import LogisticRegression
import numpy as np
import joblib

# Configuración de CUDA
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Cargar modelos preentrenados
def load_model(model_path, model_type):
    if model_type == 'resnet50':
        model = models.resnet50(pretrained=False)
    else:
        model = models.resnet152(pretrained=False)  # Asumiendo que es ResNet152
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 15)  # Número de clases que has usado
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model

model_resnet50 = load_model('models/model_1_grande.pth', 'resnet50')
model_resnet152 = load_model('models/resnet121_model.pth', 'resnet152')

# Función para extraer características de los modelos
def extract_features(model, loader):
    model.eval()
    features = []
    labels = []
    with torch.no_grad():
        for inputs, targets in loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            features.append(probabilities.cpu().numpy())
            labels.append(targets.cpu().numpy())
    return np.concatenate(features), np.concatenate(labels)

# Transformaciones y DataLoader para el conjunto de validación/pruebas
valid_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
valid_data = datasets.ImageFolder('C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/validation', transform=valid_transforms)
valid_loader = DataLoader(valid_data, batch_size=128, shuffle=False)

# Extraer características de los modelos
features_resnet50, labels = extract_features(model_resnet50, valid_loader)
features_resnet152, _ = extract_features(model_resnet152, valid_loader)

# Apilar las características horizontalmente
stacked_features = np.hstack((features_resnet50, features_resnet152))

# Entrenar el meta-modelo
meta_model = LogisticRegression(max_iter=1000)
meta_model.fit(stacked_features, labels)

# Guardar el meta-modelo
joblib.dump(meta_model, 'models/stacked_model.joblib')


['models/stacked_model.joblib']

In [6]:
import torch
import torchvision.models as models
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from sklearn.linear_model import LogisticRegression
import numpy as np
import joblib

# Configuración de CUDA
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Cargar modelos preentrenados
def load_model(model_path, model_type):
    if model_type == 'resnet50':
        model = models.resnet50(pretrained=False)
    else:
        model = models.resnet152(pretrained=False)  # Asumiendo que es ResNet152
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 15)  # Número de clases que has usado
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model

model_resnet50 = load_model('models/model_1_grande.pth', 'resnet50')
model_resnet152 = load_model('models/resnet121_model.pth', 'resnet152')

# Función para extraer características de los modelos
def extract_features(model, loader):
    model.eval()
    features = []
    labels = []
    with torch.no_grad():
        for inputs, targets in loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            features.append(probabilities.cpu().numpy())
            labels.append(targets.cpu().numpy())
    return np.concatenate(features), np.concatenate(labels)

# Transformaciones y DataLoader para el conjunto de validación/pruebas
valid_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
valid_data = datasets.ImageFolder('C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/validation', transform=valid_transforms)
valid_loader = DataLoader(valid_data, batch_size=128, shuffle=False)

# Extraer características de los modelos
features_resnet50, labels = extract_features(model_resnet50, valid_loader)
features_resnet152, _ = extract_features(model_resnet152, valid_loader)

# Apilar las características horizontalmente
stacked_features = np.hstack((features_resnet50, features_resnet152))

# Entrenar el meta-modelo
meta_model = LogisticRegression(max_iter=1000)
meta_model.fit(stacked_features, labels)

# Guardar el meta-modelo
joblib.dump(meta_model, 'models/stacked_model2.joblib')

# Evaluar el modelo
predictions = meta_model.predict(stacked_features)
accuracy = np.mean(predictions == labels)
print("Accuracy of the stacked model:", accuracy)


Accuracy of the stacked model: 0.9693333333333334
